# Exercises - Data Science Sprint 01 - M1 T01
   **Relational databases (SQL)**

These are the objectives of this Sprint:
* Construct entity-relationship and relational diagrams
* Construct a relational database with MySQL
* Perform simple queries
* Performs operations on the MySQL database

## Exercise #1

**Creation of database**

From the attached documents (structure and data), create a database with MySQL. Show the main features of the created schema and explains the different tables and variables in it.


### Database creation

In this sprint I'm going to use the Python Connector to MySQL database

In [1]:
# Import MySQL connector 
import mysql.connector
from mysql.connector import errorcode

# Establish connection to server with host, username and password
mydb = mysql.connector.connect(
    host = 'localhost',
    user='root',
    password="pass1234"
)

# Instantiates objects that can execute SQL statements
mycursor = mydb.cursor()

# Creation of database
mycursor.execute("CREATE DATABASE SGAE_Practica;")

# Create the SCHEMA movies
mycursor.execute("CREATE SCHEMA movies;")

mycursor.close()
mydb.close()

### Structure creation - Tables

In order to analyze our data structure, we can follow these steps:

1. Find Entities
  On our data structure we can find 4 entities:
    - genre
    - movie
    - role 
    - person

 
 2. Find our their relations
    - In our case we have a"awarded" relation between movie-role-person tables
        - This is a relation N:M:P, and we have to create a new relation table, named movie_person (+role)
    - Movie have a relation "genre of" with genre table
        - This is a relation 1:N, because different movies can have the same role
    - Person has a "parent" self-relation with person table
        - This is a relation 1:N with the same table, because same person can have more that one parent awarded
 
 
 3. The table attributes
 
     Genre:
        - genre_name
        - created by user
        - created_date
        - updated_date
        
     Role:
        - role_name
        - created by user
        - created_date
        - updated_date

    Person:
        - person_name
        - person_country
        - person_dob
        - person_dod
        - person_parent
        - created by user
        - created_date
        - updated_date
        
    Movie:
        - movie_title
        - movie_date
        - movie_format
        - created by user
        - created_date
        - updated_date

    Movie_Person:
        - movie_award_ind
        - created by user
        - created_date
        - updated_date

4. Definition of Primary Keys (pk) and Foreign Keys (fk)

    Genre:
        - genre_id (pk)
 
    Role:
        - role_id (pk)
 
    Person:
        - person_id (pk)
        - person_parent_id (fk) -> Self-Relation to person_id (pk)
        
    Movie:
        - movie_id (pk)
        - movie_genre_id (fk) - > Relation to genre_id

    Movie_Person:
        - movie_id + person_id + role_id (pk) -> Relation to:
            - movie_id (fk)
            - person_id (fk)
            - role_id (fk)
 

In [2]:
# Establish connection to server with host, username and password
mydb = mysql.connector.connect(
    host = 'localhost',
    user='root',
    password="pass1234",
    database = "movies"
)

# Instantiates objects that can execute SQL statements
mycursor = mydb.cursor()

TABLES = {}
TABLES ['movies.tb_genre'] = (
    "CREATE TABLE movies.tb_genre ("
    "genre_id         INTEGER NOT NULL ,"
    "genre_name       CHARACTER VARYING(40) NOT NULL ,"
    "created_by_user  CHARACTER VARYING(10) NOT NULL DEFAULT 'OS_SGAD' ,"
    "created_date     DATE ,"
    "updated_date     DATE ,"
    "CONSTRAINT pk_genre PRIMARY KEY (genre_id));"
    )

TABLES ['movies.tb_movie'] = (
    "CREATE TABLE movies.tb_movie ("
    "movie_id         INTEGER NOT NULL ,"
    "movie_title      CHARACTER VARYING(100) NOT NULL ,"
    "movie_date       DATE ,"
    "movie_format     CHARACTER VARYING(50) ,"
    "movie_genre_id   INTEGER ,"
    "created_by_user  CHARACTER VARYING(10) NOT NULL DEFAULT 'OS_SGAD' ,"
    "created_date     DATE ,"
    "updated_date     DATE ,"
    "CONSTRAINT pk_movie PRIMARY KEY (movie_id) ,"
    "CONSTRAINT fk_movie_genre FOREIGN KEY (movie_genre_id) REFERENCES movies.tb_genre (genre_id));"
    )

TABLES ['movies.tb_role'] = (
    "CREATE TABLE movies.tb_role ("
    "role_id          INTEGER NOT NULL ,"
    "role_name        CHARACTER VARYING(60) NOT NULL ,"
    "created_by_user  CHARACTER VARYING(10) NOT NULL DEFAULT 'OS_SGAD' ,"
    "created_date     DATE ,"
    "updated_date     DATE ,"
    "CONSTRAINT pk_role PRIMARY KEY (role_id));"
     )

TABLES ['movies.tb_person'] = (
    "CREATE TABLE movies.tb_person(person_id INTEGER NOT NULL ,"
    "person_name      CHARACTER VARYING(100) NOT NULL,"
    "person_country   CHARACTER VARYING(40) , "
    "person_dob       DATE NOT NULL ,"
    "person_dod       DATE ,"
    "person_parent_id INTEGER ,"
    "created_by_user  CHARACTER VARYING(10) NOT NULL DEFAULT 'OS_SGAD' ,"
    "created_date     DATE ,"
    "updated_date     DATE ,"
    "CONSTRAINT pk_person PRIMARY KEY (person_id) ,"
    "CONSTRAINT fk_person_parent FOREIGN KEY (person_parent_id) REFERENCES movies.tb_person (person_id));"
    )

TABLES ['movies.tb_movie_person'] = (
    "CREATE TABLE movies.tb_movie_person ("
    "movie_id         INTEGER NOT NULL ,"
    "person_id        INTEGER NOT NULL ,"
    "role_id          INTEGER NOT NULL ,"
    "movie_award_ind  CHAR(1) NOT NULL ,"
    "created_by_user  CHARACTER VARYING(10) NOT NULL DEFAULT 'OS_SGAD' ,"
    "created_date     DATE ,"
    "updated_date     DATE ,"
    "CONSTRAINT pk_movper PRIMARY KEY (movie_id, person_id, role_id) ,"
    "CONSTRAINT fk_movper_movie FOREIGN KEY (movie_id) REFERENCES movies.tb_movie (movie_id) ,"
    "CONSTRAINT fk_movper_person FOREIGN KEY (person_id) REFERENCES movies.tb_person (person_id) ,"
    "CONSTRAINT fk_movper_role FOREIGN KEY (role_id) REFERENCES movies.tb_role (role_id));"
    )

for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        mycursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

mycursor.close()
mydb.close()

Creating table movies.tb_genre: OK
Creating table movies.tb_movie: OK
Creating table movies.tb_role: OK
Creating table movies.tb_person: OK
Creating table movies.tb_movie_person: OK


### Insert data to tb_genre table


In [3]:
# Establish connection to server with host, username and password
mydb = mysql.connector.connect(
    host = 'localhost',
    user='root',
    password="pass1234",
    database = "movies"
    )

# Instantiates objects that can execute SQL statements
mycursor = mydb.cursor()

operation = "INSERT INTO movies.tb_genre (genre_id, genre_name) VALUES ( 1, 'Acción'); \
             INSERT INTO movies.tb_genre (genre_id, genre_name) VALUES ( 2, 'Ciencia Ficción'); \
             INSERT INTO movies.tb_genre (genre_id, genre_name) VALUES ( 3, 'Comedia'); \
             INSERT INTO movies.tb_genre (genre_id, genre_name) VALUES ( 4, 'Drama'); \
             INSERT INTO movies.tb_genre (genre_id, genre_name, created_by_user) VALUES ( 5, 'Fantasía', 'apermag'); \
             INSERT INTO movies.tb_genre (genre_id, genre_name, created_by_user, created_date, updated_date) VALUES ( 6, 'Melodrama', 'apermag', '2018-09-01', '2018-09-27'); \
             INSERT INTO movies.tb_genre (genre_id, genre_name) VALUES ( 7, 'Musical'); \
             INSERT INTO movies.tb_genre (genre_id, genre_name) VALUES ( 8, 'Romance'); \
             INSERT INTO movies.tb_genre (genre_id, genre_name) VALUES ( 9, 'Suspense'); \
             INSERT INTO movies.tb_genre (genre_id, genre_name) VALUES (10, 'Terror'); \
             INSERT INTO movies.tb_genre (genre_id, genre_name) VALUES (11, 'Bélico'); "

for result in mycursor.execute(operation, multi=True):
  if result.with_rows:
    print("Rows produced by statement '{}':".format(result.statement))
    print(result.fetchall())
  else:
    print("Number of rows affected by statement '{}': {}".format(result.statement, result.rowcount))


mydb.commit()

mycursor.close()
mydb.close()


#result = mycursor.fetchall()
#for all in result:
#  print(all)

Number of rows affected by statement 'INSERT INTO movies.tb_genre (genre_id, genre_name) VALUES ( 1, 'Acción')': 1
Number of rows affected by statement 'INSERT INTO movies.tb_genre (genre_id, genre_name) VALUES ( 2, 'Ciencia Ficción')': 1
Number of rows affected by statement 'INSERT INTO movies.tb_genre (genre_id, genre_name) VALUES ( 3, 'Comedia')': 1
Number of rows affected by statement 'INSERT INTO movies.tb_genre (genre_id, genre_name) VALUES ( 4, 'Drama')': 1
Number of rows affected by statement 'INSERT INTO movies.tb_genre (genre_id, genre_name, created_by_user) VALUES ( 5, 'Fantasía', 'apermag')': 1
Number of rows affected by statement 'INSERT INTO movies.tb_genre (genre_id, genre_name, created_by_user, created_date, updated_date) VALUES ( 6, 'Melodrama', 'apermag', '2018-09-01', '2018-09-27')': 1
Number of rows affected by statement 'INSERT INTO movies.tb_genre (genre_id, genre_name) VALUES ( 7, 'Musical')': 1
Number of rows affected by statement 'INSERT INTO movies.tb_genre (g

### Insert data to tb_movie table


In [4]:
# Establish connection to server with host, username and password
mydb = mysql.connector.connect(
    host = 'localhost',
    user='root',
    password="pass1234",
    database = "movies"
    )

# Instantiates objects that can execute SQL statements
mycursor = mydb.cursor()

sql = "INSERT INTO movies.tb_movie (movie_id, movie_title, movie_date, movie_format, movie_genre_id) \
       VALUES (%s, %s, %s, %s, %s);"
data = [( 1, 'Apocalypse Now', '1979-05-10', 'Film', 11),
        ( 2, 'Star Wars:Episode IV - A New Hope', '1977-05-25', 'Film', 2),
        ( 3, 'Indiana Jones and the Temple of Doom', '1984-05-08', 'Film', 1),
        ( 4, 'The Terminal', '2004-06-18', 'Digital', 3),
        ( 5, 'Jaws', '1975-01-01', 'Film', 10),
        ( 6, 'ET The Extraterrestrial', '1982-07-25', 'Film', 5),
        ( 7, 'Psycho', '1960-05-06', 'Film', 9),
        ( 8, 'Ocho Apellidos Vascos', '2014-03-14', 'Digital', 3),
        ( 9, 'Ocho Apellidos Catalanes', '2016-06-09', 'Digital', 8),
        (10, 'El otro lado de la cama', '2002-09-04', 'Digital', 8),
        (11, 'La Gran Familia Española', '2012-10-15', 'Digital', 3),
        (12, 'El dia de la bestia', '1994-12-25', 'Film', 1),
        (13, 'Braveheart', '1995-08-08', 'Film', 4),
        (14, 'The Shawshank Redemption', '1992-01-07', 'Film', 4),
        (15, 'Las brujas de Zugarramurdi', '2009-10-07', 'Digital', 9),
        (16, 'Blade Runner', '1982-12-25', 'Digital', 2)]


mycursor.executemany(sql, data)

mydb.commit()

mycursor.close()
mydb.close()

### Insert data to tb_role table


In [5]:
# Establish connection to server with host, username and password
mydb = mysql.connector.connect(
    host = 'localhost',
    user='root',
    password="pass1234",
    database = "movies"
    )

# Instantiates objects that can execute SQL statements
mycursor = mydb.cursor()

sql = "INSERT INTO movies.tb_role (role_id, role_name) VALUES (%s, %s);"

data = [( 1, 'Actor'),
        ( 2, 'Director'),
        ( 3, 'Productor'),
        ( 4, 'Guionista'),
        ( 5, 'Música')]

mycursor.executemany(sql, data)

mydb.commit()

mycursor.close()
mydb.close()

### Insert data to tb_person table


In [6]:
# Establish connection to server with host, username and password
mydb = mysql.connector.connect(
    host = 'localhost',
    user='root',
    password="pass1234",
    database = "movies"
    )

# Instantiates objects that can execute SQL statements
mycursor = mydb.cursor()

sql = "INSERT INTO movies.tb_person (person_id, person_name, person_country, person_dob, person_dod, person_parent_id) \
       VALUES (%s, %s, %s, %s, %s, %s);"

data = [( 1, 'Francis Ford Coppola', 'United States', '1939-04-07',None, None),
        ( 2, 'Carmine Coppola', 'United States', '1945-07-08', None, None),
        ( 3, 'Marlon Brando', 'United States', '1924-04-03', '2004-07-01', None),
        ( 4, 'Robert Duvall', 'United States', '1931-01-05', None, None),
        ( 5, 'Martin Sheen', 'United States', '1940-08-03', None, None),
        ( 6, 'Harrison Ford', 'United States', '1942-07-13', None, None),
        ( 7, 'George Lucas', 'United States', '1944-05-14', None, None),
        ( 8, 'Gary Kurtz', 'United States', '1940-07-27', None, None),
        ( 9, 'Steven Spielberg', 'United States', '1946-12-18', None, None),
        (10, 'John Williams', 'United States', '1928-08-08', None, None),
        (11, 'Tom Hanks', 'United States', '1956-07-09', None, None),
        (12, 'Catherine Zeta-Jones', 'Wales', '1969-09-25', None, None),
        (13, 'Alfred Joseph Hitchcock', 'United Kingdom', '1899-08-13', '1980-04-29', None),
        (14, 'Anthony Perkins', 'United States', '1934-04-04', '1992-09-08', None),
        (15, 'Vera Miles', 'United States', '1929-08-23', None, None),
        (16, 'Emilio Martinez Lazaro', 'Spain', '1956-09-09', None, None),
        (17, 'Dani Rovira', 'Spain', '1984-07-01', None, None),
        (18, 'Clara Lago', 'Spain', '1986-04-17', None, None),
        (19, 'Carmen Machi', 'Spain', '1964-08-09', None, None),
        (20, 'Karra Elejalde', 'Spain', '1960-03-06', None, None),
        (21, 'Daniel Sanchez Arevalo', 'Spain', '1970-06-08', None, None),
        (22, 'Quim Gutierrez', 'Spain', '1981-03-27', None, None),
        (23, 'Robert Alamo', 'Spain', '1970-05-06', None, None),
        (24, 'Hector Colome', 'Spain', '1944-10-25', '2015-02-28', None),
        (25, 'Veronica Echegui', 'Spain', '1983-03-14', None, None),
        (26, 'Patrick Criado', 'Spain', '1995-09-23', None, None),
        (27, 'Sean Connery', 'Scotland', '1930-07-08', None, None),
        (28, 'Mel Gibson', 'Australia', '1950-08-09', None, None),
        (29, 'Morgan Freeman', 'United States', '1935-10-01', None, None),
        (30, 'Tim Robbins', 'United States', '1949-06-07', None, None),
        (41, 'Charlie Sheen', 'United States', '1965-09-03', None, 5),
        (42, 'Emilio Estevez', 'United States', '1962-05-12', None, 5),
        (43, 'Ramón Estevez', 'United States', '1963-08-07', None, 5),
        (44, 'Reneé Estevez', 'United States', '1967-04-02', None, 5),
        (45, 'Paula Speert Sheen', 'United States', '1986-01-06', None,41),
        (46, 'Bob Sheen', 'United States', '2009-05-01', None,41),
        (47, 'Max Sheen', 'United States', '2009-05-01', None,41),
        (48, 'Sam Sheen', 'United States', '2004-03-09', None,41),
        (49, 'Lola Sheen', 'United States', '2005-06-01', None,41),
        (50, 'Paula Jones-Sheen', 'United States', '2003-07-06', None,45),
        (51, 'Paloma Rae Estevez', 'United States', '1986-02-15', None,42),
        (52, 'Taylor Levi Estevez', 'United States', '1984-06-22', None,42)]

mycursor.executemany(sql, data)

mydb.commit()

mycursor.close()
mydb.close()


### Insert data to tb_movie_person table


In [7]:
# Establish connection to server with host, username and password
mydb = mysql.connector.connect(
    host = 'localhost',
    user='root',
    password="pass1234",
    database = "movies"
    )

# Instantiates objects that can execute SQL statements
mycursor = mydb.cursor()

sql = "INSERT INTO movies.tb_movie_person (movie_id, person_id, role_id, movie_award_ind) VALUES (%s, %s, %s, %s);"

data = [( 1, 1, 2, 'Y'),
        ( 1, 1, 3, 'N'),
        ( 1, 1, 5, 'N'),
        ( 1, 2, 5, 'N'),
        ( 1, 3, 1, 'N'),
        ( 1, 4, 1, 'N'),
        ( 1, 5, 1, 'Y'),
        ( 1,41, 1, 'N'),
        ( 1, 6, 1, 'N'),
        ( 2, 6, 1, 'N'),
        ( 2, 7, 2, 'Y'),
        ( 2, 8, 3, 'N'),
        ( 3, 6, 1, 'N'),
        ( 3, 7, 4, 'N'),
        ( 3, 9, 2, 'N'),
        ( 3,10, 5, 'N'),
        ( 4, 9, 2, 'N'),
        ( 4, 9, 3, 'N'),
        ( 4,11, 1, 'N'),
        ( 4,12, 1, 'N'),
        ( 5, 9, 2, 'N'),
        ( 6, 9, 2, 'N'),
        ( 7,13, 1, 'N'),
        ( 7,13, 2, 'N'),
        ( 7,13, 3, 'N'),
        ( 7,14, 2, 'N'),
        ( 7,15, 2, 'N'),
        ( 3, 7, 1, 'N'),
        ( 8,16, 2, 'N'),
        ( 9,16, 2, 'N'),
        ( 8,17, 1, 'N'),
        ( 8,18, 1, 'N'),
        ( 8,19, 1, 'N'),
        ( 8,20, 1, 'N'),
        ( 9,17, 1, 'N'),
        ( 9,18, 1, 'N'),
        ( 9,19, 1, 'N'),
        ( 9,20, 1, 'N'),
        (10,16, 2, 'N'),
        (11,21, 2, 'N'),
        (11,21, 4, 'N'),
        (11,22, 1, 'N'),
        (11,23, 1, 'N'),
        (11,24, 1, 'N'),
        (11,25, 1, 'N'),
        (11,26, 1, 'N'),
        (13,28, 1, 'Y'),
        (13,28, 2, 'N'),
        (14,29, 1, 'N'),
        (14,30, 1, 'N')]

mycursor.executemany(sql, data)

mydb.commit()

mycursor.close()
mydb.close()

### Function definition to execute sql statements 


In [8]:
# This funcion receives 2 parametres, the first one is the sql statement
# The second one is a boolean value, True means that we have to execute commit because database has been altered and
# must be update.

def sql_function(sql, commit_yn):
# Establish connection to server with host, username and password
    mydb = mysql.connector.connect(
        host = 'localhost',
        user='root',
        password="pass1234",
        database = "movies"
        )

    # Instantiates objects that can execute SQL statements
    mycursor = mydb.cursor()

    mycursor.execute(sql)
    
    if (commit_yn):
        mydb.commit()
    else:
        myresult = mycursor.fetchall()
        for x in myresult:
            print(x)

    mycursor.close()

    mydb.close()

## Exercise #2

**Perform query**

Obtain the name, country and date of birth of those persons for whom there is no date of death, and order the data from the oldest person to the youngest person.

In [9]:
# Exercise 2
# Statement definition
sql = "SELECT person_name, person_country, person_dob FROM tb_person \
       WHERE person_dod is NULL \
       ORDER BY person_dob ASC;"

# Call to function to execute statement and show results
sql_function(sql, False)


('John Williams', 'United States', datetime.date(1928, 8, 8))
('Vera Miles', 'United States', datetime.date(1929, 8, 23))
('Sean Connery', 'Scotland', datetime.date(1930, 7, 8))
('Robert Duvall', 'United States', datetime.date(1931, 1, 5))
('Morgan Freeman', 'United States', datetime.date(1935, 10, 1))
('Francis Ford Coppola', 'United States', datetime.date(1939, 4, 7))
('Gary Kurtz', 'United States', datetime.date(1940, 7, 27))
('Martin Sheen', 'United States', datetime.date(1940, 8, 3))
('Harrison Ford', 'United States', datetime.date(1942, 7, 13))
('George Lucas', 'United States', datetime.date(1944, 5, 14))
('Carmine Coppola', 'United States', datetime.date(1945, 7, 8))
('Steven Spielberg', 'United States', datetime.date(1946, 12, 18))
('Tim Robbins', 'United States', datetime.date(1949, 6, 7))
('Mel Gibson', 'Australia', datetime.date(1950, 8, 9))
('Tom Hanks', 'United States', datetime.date(1956, 7, 9))
('Emilio Martinez Lazaro', 'Spain', datetime.date(1956, 9, 9))
('Karra Elejal

## Exercise #3

**Perform query**

Get the genre name and the total number of movies in that genre, and sort it in descending order of total number of movies.

In [10]:
# Exercise 3
# Statement definition
sql = "SELECT COUNT(genre_name) AS gn_count, genre_name FROM tb_movie \
       INNER JOIN tb_genre WHERE tb_genre.genre_id = tb_movie.movie_genre_id \
       GROUP BY genre_name \
       ORDER BY gn_count DESC"

# Call to function to execute statement and show results
sql_function(sql, False)

(3, 'Comedia')
(2, 'Acción')
(2, 'Ciencia Ficción')
(2, 'Drama')
(2, 'Romance')
(2, 'Suspense')
(1, 'Fantasía')
(1, 'Terror')
(1, 'Bélico')


## Exercise #4

**Perform queries**

### Section 1

Get, for each person, their name and the maximum number of different roles they have had in the same movie.

In [11]:
# Exercise 4.1
# Statement definition
sql =  "SELECT COUNT(tb_role.role_id) AS role_count, tb_person.person_name, tb_movie.movie_title \
        FROM (((tb_movie_person \
        INNER JOIN tb_role ON tb_role.role_id = tb_movie_person.role_id) \
        INNER JOIN tb_person ON tb_person.person_id = tb_movie_person.person_id) \
        INNER JOIN tb_movie ON tb_movie.movie_id = tb_movie_person.movie_id) \
        GROUP BY movie_title, person_name \
        ORDER BY role_count;"

# Call to function to execute statement and show results
sql_function(sql, False)

(1, 'Marlon Brando', 'Apocalypse Now')
(1, 'Robert Duvall', 'Apocalypse Now')
(1, 'Martin Sheen', 'Apocalypse Now')
(1, 'Harrison Ford', 'Apocalypse Now')
(1, 'Charlie Sheen', 'Apocalypse Now')
(1, 'Harrison Ford', 'Star Wars:Episode IV - A New Hope')
(1, 'Harrison Ford', 'Indiana Jones and the Temple of Doom')
(1, 'Carmine Coppola', 'Apocalypse Now')
(1, 'Tom Hanks', 'The Terminal')
(1, 'Catherine Zeta-Jones', 'The Terminal')
(1, 'John Williams', 'Indiana Jones and the Temple of Doom')
(1, 'Dani Rovira', 'Ocho Apellidos Vascos')
(1, 'Clara Lago', 'Ocho Apellidos Vascos')
(1, 'Carmen Machi', 'Ocho Apellidos Vascos')
(1, 'Karra Elejalde', 'Ocho Apellidos Vascos')
(1, 'Dani Rovira', 'Ocho Apellidos Catalanes')
(1, 'Clara Lago', 'Ocho Apellidos Catalanes')
(1, 'Carmen Machi', 'Ocho Apellidos Catalanes')
(1, 'Karra Elejalde', 'Ocho Apellidos Catalanes')
(1, 'Quim Gutierrez', 'La Gran Familia Española')
(1, 'Robert Alamo', 'La Gran Familia Española')
(1, 'Hector Colome', 'La Gran Familia Es

### Section 2

Show those people who have assumed more than one role in the same film.

In [12]:
# Exercise 4.2
# Statement definition
sql =  "SELECT COUNT(tb_role.role_id) AS role_count, tb_person.person_name, tb_movie.movie_title \
        FROM (((tb_movie_person \
        INNER JOIN tb_role ON tb_role.role_id = tb_movie_person.role_id) \
        INNER JOIN tb_person ON tb_person.person_id = tb_movie_person.person_id) \
        INNER JOIN tb_movie ON tb_movie.movie_id = tb_movie_person.movie_id) \
        GROUP BY movie_title, person_name \
        HAVING role_count > 1 \
        ORDER BY role_count DESC;"

# Call to function to execute statement and show results
sql_function(sql, False)

(3, 'Alfred Joseph Hitchcock', 'Psycho')
(3, 'Francis Ford Coppola', 'Apocalypse Now')
(2, 'George Lucas', 'Indiana Jones and the Temple of Doom')
(2, 'Mel Gibson', 'Braveheart')
(2, 'Steven Spielberg', 'The Terminal')
(2, 'Daniel Sanchez Arevalo', 'La Gran Familia Española')


## Execise #5

**Perform queries**

You must create a new genre called "Documental" which has the number 69 as its identifier.

In [13]:
# Exercise 5
# Statement definition
sql = "INSERT INTO tb_genre (genre_id, genre_name) VALUES (69, 'Documental');"

# Call to function to execute statement and show results
sql_function(sql, True)

In [14]:
# EXERCICI 5 (CONT)
# Statement definition
sql = "SELECT * FROM tb_genre;"

# Call to function to execute statement and show results
sql_function(sql, False)

(1, 'Acción', 'OS_SGAD', None, None)
(2, 'Ciencia Ficción', 'OS_SGAD', None, None)
(3, 'Comedia', 'OS_SGAD', None, None)
(4, 'Drama', 'OS_SGAD', None, None)
(5, 'Fantasía', 'apermag', None, None)
(6, 'Melodrama', 'apermag', datetime.date(2018, 9, 1), datetime.date(2018, 9, 27))
(7, 'Musical', 'OS_SGAD', None, None)
(8, 'Romance', 'OS_SGAD', None, None)
(9, 'Suspense', 'OS_SGAD', None, None)
(10, 'Terror', 'OS_SGAD', None, None)
(11, 'Bélico', 'OS_SGAD', None, None)
(69, 'Documental', 'OS_SGAD', None, None)


## Execise #6

**Perform queries**

Remove the movie "La Gran Familia Española" from the database.

In this exercise, due to the the Referential Integrity is not possible to remove only the row with "La Gran Familia Española" movie. First we have to modify the constrain in order to add "ON DELETE CASCADE". With this new SQL statement, on deleting the movie it will delete other table rows in order to maintain the Referential Integrity on database. 

In [15]:
# Exercise 6
# Statement definition
sql = "ALTER TABLE tb_movie_person DROP CONSTRAINT fk_movper_movie; \
       ALTER TABLE tb_movie_person \
       ADD CONSTRAINT fk_movper_movie FOREIGN KEY (movie_id) REFERENCES movies.tb_movie (movie_id) ON DELETE CASCADE;"

# Call to function to execute statement and show results
sql_function(sql, False)

In [16]:
# Exercise 6 (CONT-1)
# Statement definition
sql = "DELETE FROM tb_movie WHERE movie_title = 'La Gran Familia Española';"

# Call to function to execute statement and show results
sql_function(sql, True)

In [17]:
# Exercise 6 (CONT-2)
# Statement definition
sql = "SELECT * FROM tb_movie;"

# Call to function to execute statement and show results
sql_function(sql, False)

(1, 'Apocalypse Now', datetime.date(1979, 5, 10), 'Film', 11, 'OS_SGAD', None, None)
(2, 'Star Wars:Episode IV - A New Hope', datetime.date(1977, 5, 25), 'Film', 2, 'OS_SGAD', None, None)
(3, 'Indiana Jones and the Temple of Doom', datetime.date(1984, 5, 8), 'Film', 1, 'OS_SGAD', None, None)
(4, 'The Terminal', datetime.date(2004, 6, 18), 'Digital', 3, 'OS_SGAD', None, None)
(5, 'Jaws', datetime.date(1975, 1, 1), 'Film', 10, 'OS_SGAD', None, None)
(6, 'ET The Extraterrestrial', datetime.date(1982, 7, 25), 'Film', 5, 'OS_SGAD', None, None)
(7, 'Psycho', datetime.date(1960, 5, 6), 'Film', 9, 'OS_SGAD', None, None)
(8, 'Ocho Apellidos Vascos', datetime.date(2014, 3, 14), 'Digital', 3, 'OS_SGAD', None, None)
(9, 'Ocho Apellidos Catalanes', datetime.date(2016, 6, 9), 'Digital', 8, 'OS_SGAD', None, None)
(10, 'El otro lado de la cama', datetime.date(2002, 9, 4), 'Digital', 8, 'OS_SGAD', None, None)
(12, 'El dia de la bestia', datetime.date(1994, 12, 25), 'Film', 1, 'OS_SGAD', None, None)
(13

## Execise #7

**Perform queries**

Change the genre of the film "Ocho apellidos catalanes" so that it is recorded as a comedy and not as a romance.

In [18]:
# Exercise 7
# Statement definition
sql = "UPDATE tb_movie \
       SET movie_genre_id = 3 \
       WHERE movie_title = 'Ocho Apellidos Catalanes';"

# Call to function to execute statement and show results
sql_function(sql, True)

In [19]:
# Exercise 7 (CONT)
# Statement definition
sql = "SELECT movie_title, movie_genre_id FROM tb_movie \
       WHERE movie_title = 'Ocho Apellidos Catalanes';"

# Call to function to execute statement and show results
sql_function(sql, False)

('Ocho Apellidos Catalanes', 3)
